In [1]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('data/chat_history.db')
cursor = conn.cursor()

# Execute a query to retrieve all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the list of tables
print("Tables in the database:")
for table in tables:
    print(table[0])



Tables in the database:
conversation
conversation_record
<class 'AG.ConversationRecord'>
conversation_history_T
conversations_history_table_sqlalchemy


modal volume get db_data /chat_history.db . --force


In [3]:
import pandas as pd

# Set display options to avoid truncation
pd.set_option('display.max_colwidth', None)  # No limit on column width
pd.set_option('display.max_rows', None)      # Display all rows if needed
# Read the 'conversation' table into a pandas DataFrame
df_conversation = pd.read_sql_query("SELECT * FROM conversations_history_table_sqlalchemy", conn)


df_conversation.head(110)


,message_id,session_id,role,content,created_at
0,c591e890-a4bd-4656-aabd-64aba1df8590,95bbb3dc-9356-4af8-b781-feda4afe65d5,user,hi sqlalchemy,2024-11-12 16:26:35.854878
1,319c5ed1-5995-4493-8d0c-ebf335e2cbb7,42e492e4-98f3-465f-8848-5afddd049d96,user,ok,2024-11-12 16:27:09.641403
2,e733da37-982d-4ff6-a938-02b1d6dc0d77,db418e4b-5df2-401e-aba1-2cfae12129e8,user,Hello Session ID: 9cfa190d-0cd2-48d8-9673-abd9...,2024-11-13 11:49:36.979689
3,10538ce5-77ec-4a0e-b22f-a65f41dbb1a4,e2399db7-a76f-4313-86f1-a4cf997eed87,user,Hello Session ID: 9cfa190d-0cd2-48d8-9673-abd9...,2024-11-13 11:50:10.632955
4,5d971fa4-1210-43ba-9006-e928060752f7,099a288d-32df-4b34-a9db-f8fa4b621719,user,Hello Session ID: 9cfa190d-0cd2-48d8-9673-abd9...,2024-11-13 11:50:19.656492
5,539fb865-fbd2-45d7-a049-62e3f84ed29c,1da0334d-a664-463c-82de-a7b200a81bc2,user,Hi Session ID: ee73159c-2573-491a-a7e2-0c32fe7...,2024-11-13 11:51:52.273854
6,8f456b2b-35ae-4b82-8048-d595fd4cbae8,d41f6c89-e9e8-4890-a077-3b2ec444da79,user,round 2: Session ID: ad67dd22-3089-41c8-8597-a...,2024-11-13 12:08:18.573401
7,51f2aa4c-7d33-4545-835e-7d98224a9235,fdf54815-94e8-4d09-a538-208cde7a3791,user,round 2: Session ID: ad67dd22-3089-41c8-8597-a...,2024-11-13 12:08:52.887376
8,ad4667a8-ec27-4d83-a400-ed4665ea6351,aa72e831-44fe-44af-9977-875260d4f1f2,user,now I will refresh... if you dont see my next ...,2024-11-13 12:09:21.249517
9,3aeb064f-c394-4d67-be84-99e75dc19fba,8329dafa-b1e0-4e05-a0f4-a6710ad039bb,user,ok ru paul,2024-11-13 12:53:13.141200


In [7]:
df_conversation.to_csv('df_conversation.csv', index=False)


In [5]:
session_id = '65c01756-9252-4a9d-913c-d12b3dc5310d'

df_conversation_filt = pd.read_sql_query(
    f"SELECT * FROM conversation_history_T WHERE session_id = '{session_id}'", 
    conn
)
df_conversation_filt.head()

,id,session_id,role,content,created_at
0,aa701f1c-fa8c-4976-ad6d-b0f87744f572,65c01756-9252-4a9d-913c-d12b3dc5310d,user,hI,2024-11-10 13:45:06
1,afe3566a-e586-4fdb-96e4-cd94a6f2e01d,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,HHello! What’s been going on that brings you h...,2024-11-10 13:45:49
2,ae6294e3-b127-4dcc-a7e7-bea9dbf8fada,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,It seems like things have become overwhelming...,2024-11-11 12:00:01
3,3886dbea-d45f-47d5-a575-d97938182a31,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,That sounds quite frustrating.I'd love to exp...,2024-11-11 12:02:13


In [7]:
db_path = 'data/chat_history.db'
session_id = '65c01756-9252-4a9d-913c-d12b3dc5310d'

In [8]:
def fetch_conversation_history(session_id, db_path):
    conn = sqlite3.connect(db_path)
    query = """
    SELECT role, content FROM conversation_history_T
    WHERE session_id = ?
    ORDER BY created_at ASC
    """
    # Fetch all messages for the session
    df_conversation_filt = pd.read_sql_query(query, conn, params=(session_id,))
    conn.close()
    
    # Accumulate all messages into a single conversation history string
    conversation = ''
    for _, row in df_conversation_filt.iterrows():
        role, content = row['role'], row['content']
        message_text = f"{role.capitalize()}: {content}\n"
        conversation += message_text  # Accumulate in chronological order

    return conversation

In [12]:
fetch_conversation_history(session_id, db_path)

'User: hI\nAssistant: HHello! What’s been going on that brings you here today?\n'

In [10]:
fetch_conversation_history(session_id, db_path)

'User: hI\nAssistant: HHello! What’s been going on that brings you here today?\n'

In [5]:
# (Optional) Close the cursor if you're done with it
cursor.close()
